In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.patches import Polygon
from scipy.spatial import ConvexHull

import itertools
from sklearn.metrics import silhouette_score
from scipy.stats import f_oneway, ttest_rel
from statsmodels.stats.multitest import multipletests
from statsmodels.stats.anova import AnovaRM


import os,sys
sys.path.append(r'{}'.format(os.path.abspath(os.pardir)))

from wholebrain_tools import aba, genes, dataIO
import wholebrain_tools.stats as st 
import wholebrain_tools.graphics as gt

# Instantiate an Atlas object from the aba module
# The first time you run this it will download the structures.json file from the Allen Institute server
paths = dataIO.pathParser()

nodes_path = paths.structures
A = aba.Atlas(nodes = nodes_path)
DFM = aba.AnatomyDataFrameManager(A)

# PV energy in cortical subnetworks
-----

### Load data

In [ ]:
# --------------------------------------------------------------------
searchPath = paths.alldata
channelName = 'pv'     # 'wfa' or 'pv'
# --------------------------------------------------------------------

df = dataIO.allMiceRegions(searchPath=searchPath, channelName=channelName, normCellIntens=True)
df = DFM.multiIndexDf_from_fineDf(df, verbose=True)

# Dataframe at mid-ontology
midDf_wfa = DFM.regionsDf_to_mid(df, verbose=False, normalize=True)

# Select only cortical areas
cortex_df_pv = midDf_wfa.loc[315]

# Calculate mean and sem
mean = cortex_df_pv.groupby('params', axis=1).mean()
sem = cortex_df_pv.groupby('params', axis=1).sem().add_suffix('_sem')

# Get the annotation of cortical areas in subnetworks
subnet = dataIO.corticalAreasGroups(toDataFrame=False)
subnet_df = pd.DataFrame.from_dict(subnet,orient='index').rename(columns={0:'function'})
subnet_df.index = A.acronyms_to_ids(subnet_df.index)

data = pd.concat([mean, sem, subnet_df], axis=1)

# Drop areas with unknown function ("other")
data = data.drop(data[data['function']=='other'].index)

# Rename values in "function" columns for aesthetic purposes
switchDict = {
    'mAssociation':'Medial Association',
    'mPrefrontal': 'Medial Prefrontal',
    'lateral':'Lateral',
    'motorSS':'Motor-Somatosensory',
    'audioVisual':'Audio-Visual'
}
data['function_nice'] = [switchDict[x] for x in data['function']]

data.head()

### Plot cortical subnetworks

In [ ]:
sns.set_style('white')

f, ax = plt.subplots(figsize=(7,7))

# Main scatterplot
g = sns.scatterplot(
    data=data,
    x='diffuseFluo',
    y='energy',
    s=140,
    hue='function_nice'
)

# Add the convex hull background
bgPalette = sns.color_palette("pastel")
for i, areacategory in enumerate(data['function'].unique()):
    set = data.loc[data['function']==areacategory, ['diffuseFluo', 'energy']]
    points = set.values
    hull = ConvexHull(points)
    p = Polygon(points[hull.vertices,:], alpha=0.3, zorder=-20, facecolor=bgPalette[i])
    ax.add_patch(p)

# Plot the errorbars
plt.errorbar(
    x=data['diffuseFluo'],
    y=data['energy'],
    xerr=data['diffuseFluo_sem'],
    yerr=data['energy_sem'],
    fmt='None',
    ecolor='gray',
    elinewidth=.5,
    zorder=-13
)

plt.legend(
    title='Cortical Subnetwork',
    fontsize=14,
    loc='best',
    title_fontsize=16,
    frameon=False
)

# Customize Axis
g.set_xlabel("PV Diffuse Fluorescence (A.U.)", fontsize=22)
g.set_ylabel("PV Energy (A.U.)", fontsize=22)
ax.tick_params(labelsize=16)

sns.despine()

# plt.savefig('CorticalSubnetworks_PV.svg',bbox_inches="tight")

### Compute PV energy for the high-WFA and the low-WFA group

In [ ]:
# Extract raw data for the cortex
cortex_df_pv = df.loc[315].copy()

# Addign each area of the cortex to the high-WFA or low-WFA groups
midids = A.ids_to_acronyms(cortex_df_pv.index.get_level_values('mid'))
functional_group = [subnet[area] for area in midids]
cortex_df_pv.loc[:,'function'] = functional_group
group = ['high-WFA' if x in ['motorSS','audioVisual','mAssociation'] else 'low-WFA' if x in ['lateral','mPrefrontal'] else 'other' for x in cortex_df_pv['function']]
cortex_df_pv['group'] = group
cortex_df_pv.set_index('group', inplace=True)
cortex_df_pv.drop('function', axis='columns', inplace=True)

#aggregate data by groups
grouped = cortex_df_pv.groupby(by='group', axis=0).sum()
#Calculate energy
areaMm = grouped.xs('areaMm2',axis=1,level='params')
fluoCellSum = grouped.xs('fluoCellSum',axis=1,level='params')
energy = fluoCellSum.divide(areaMm)
# Normalize
brainAreaMm = df.xs('areaMm2', axis=1, level='params').sum()
brainAreaPx = df.xs('areaPx', axis=1, level='params').sum()
brainCellIntensity = df.xs('fluoCellSum', axis=1, level='params').sum()
brainEnergy = brainCellIntensity.divide(brainAreaMm)
energy = energy.divide(brainEnergy)

# Drop 'other' subnetwork

energy  = energy.drop('other')


### Plot PV energy

In [ ]:
melted_en = energy.melt(ignore_index=False).reset_index()
# Define colors
cmap = cm.get_cmap("OrRd")
cmap = cm.get_cmap(cmap)
pointColor = cmap(0.85)
paletteBars = {
'high-WFA':cmap(0.6),
'low-WFA':cmap(0.2),
}


f, ax = plt.subplots(figsize=(2,7))
# Plot histograms
g = sns.barplot(data = melted_en,
            x = 'group',
            y = 'value', 
            palette=paletteBars,
            errorbar='se'
            )
# plot single animals
ax.plot(
        energy,
        linewidth=1,
        color=pointColor,
        marker='o',
        markersize=7,
        alpha=.5,
    )

# change labels
g.set_ylabel("PV Energy (A.U.)", fontdict={'fontsize':25})
g.set_xlabel("",  fontdict={'fontsize':20})
ax.tick_params(labelsize=20)
ax.set_xticklabels(ax.xaxis.get_ticklabels(), rotation=45, ha='right',fontdict={'fontsize':20})
sns.despine()

# performed paired ttest
t,p = ttest_rel(
        energy.loc['high-WFA'],
        energy.loc['low-WFA'],
        nan_policy='omit'
)
print(f"T-test - T: {t:.4f} - p-value:{p:.8f}")

# plt.savefig('pvEnergyHigh-Low.svg', bbox_inches="tight")

# PNN-PV colocalization in cortical subnetworks
----

### load data

In [ ]:
# --------------------------------------------------------------------
searchPath = paths.alldata
# --------------------------------------------------------------------

df_dots = dataIO.allMiceDots(searchPath=searchPath)
df_dots = DFM.multiIndexDf_from_dotsDf(df_dots, verbose=True)
#select cortical areas
df_dots  = df_dots.loc[315]


### Compute colocalization metrics for high-WFA and low WFA subnetworks

In [ ]:
# Assign each area to the appropriate subnetwork
midareas = A.ids_to_acronyms(df_dots.index.get_level_values('mid'))
df_dots['subnet'] = [subnet[area] for area in midareas]
df_dots['group'] = ['high-WFA' if x in ['motorSS','audioVisual','mAssociation'] else 'low-WFA' if x in ['lateral','mPrefrontal'] else 'other' for x in df_dots['subnet']]
df_dots = df_dots.drop('subnet', axis='columns', level=0)
#compute aggregated metrics
df_dots = df_dots.groupby('group').sum()
coloc_pv = df_dots.xs('n_colocalized',axis = 'columns', level = 'params')
tot_pv = df_dots.xs('n_pv',axis = 'columns', level = 'params')
wfaPositive_pv  = coloc_pv.divide(tot_pv) *100
#drop 'other' subnetwork
wfaPositive_pv = wfaPositive_pv.drop('other')
#melt df for plotting
melted_coloc = wfaPositive_pv.melt(ignore_index=False).reset_index()


### Plot colocalization

In [ ]:
# Define colors
cmap = cm.get_cmap("pink_r")
cmap = cm.get_cmap(cmap)
pointColor = cmap(0.85)
barColor = cmap(0.4)
paletteBars = {
'high-WFA':cmap(0.6),
'low-WFA':cmap(0.2),
    }

f, ax = plt.subplots(figsize=(2,7))
# plot histograms
g = sns.barplot(data = melted_coloc,
            x = 'group',
            y = 'value', 
            palette=paletteBars,
            errorbar='se'
            )
#plot single animals
ax.plot(
        wfaPositive_pv,
        linewidth=1,
        color=pointColor,
        marker='o',
        markersize=7,
        alpha=.5,
    )

#add labels
g.set_ylabel("WFA$^+$ PV cells (%)", fontdict={'fontsize':25})
g.set_xlabel("",  fontdict={'fontsize':20})
ax.tick_params(labelsize=20)
ax.set_xticklabels(ax.xaxis.get_ticklabels(), rotation=45, ha='right',fontdict={'fontsize':20})
sns.despine()

#Perform paired ttest
t,p = ttest_rel(
        wfaPositive_pv.loc['high-WFA'],
        wfaPositive_pv.loc['low-WFA'],
        nan_policy='omit'
)
print(f"T-test - T: {t:.4f} - p-value:{p:.7f}")

# plt.savefig('colocHigh-Low.svg', bbox_inches="tight")